## 『本次練習內容』
#### 運用這幾天所學觀念搭建一個CNN分類器

## 『本次練習目的』
  #### 熟悉CNN分類器搭建步驟與原理
  #### 學員們可以嘗試不同搭法，如使用不同的Maxpooling層，用GlobalAveragePooling取代Flatten等等

In [2]:
from keras.models import Sequential
from keras.layers import Convolution2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import BatchNormalization
from keras.datasets import cifar10
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import OneHotEncoder


Using TensorFlow backend.
C:\Users\a0923\Anaconda3\envs\tensorflow114python36\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\a0923\Anaconda3\envs\tensorflow114python36\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\a0923\Anaconda3\envs\tensorflow114python36\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])

In [3]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

print(x_train.shape) #(50000, 32, 32, 3)

## Normalize Data
def normalize(X_train,X_test):
        mean = np.mean(X_train,axis=(0,1,2,3))
        std = np.std(X_train, axis=(0, 1, 2, 3))
        X_train = (X_train-mean)/(std+1e-7)
        X_test = (X_test-mean)/(std+1e-7) 
        return X_train, X_test,mean,std
    
    
## Normalize Training and Testset    
x_train, x_test,mean_train,std_train = normalize(x_train, x_test) 

(50000, 32, 32, 3)


In [4]:
## OneHot Label 由(None, 1)-(None, 10)
## ex. label=2,變成[0,0,1,0,0,0,0,0,0,0]
one_hot=OneHotEncoder()
y_train=one_hot.fit_transform(y_train).toarray()
y_test=one_hot.transform(y_test).toarray()

In [5]:

classifier=Sequential()

#卷積組合
classifier.add(Convolution2D(32, 3, input_shape=(32,32,3), activation='relu', padding='same'))#32,3,3,input_shape=(32,32,3),activation='relu''
classifier.add(BatchNormalization( momentum=0.99, epsilon=0.001))

'''自己決定MaxPooling2D放在哪裡'''
#classifier.add(MaxPooling2D(pool_size=(2,2)))
classifier.add(MaxPooling2D(pool_size=(2,2)))

#卷積組合
classifier.add(Convolution2D(64, 3, activation='relu', padding='same'))
classifier.add(BatchNormalization( momentum=0.99, epsilon=0.001))

#flatten
classifier.add(Flatten())

#FC
classifier.add(Dense(100, activation='relu')) #output_dim=100,activation=relu

#輸出
classifier.add(Dense(10,activation='softmax'))

#超過兩個就要選categorical_crossentrophy
classifier.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
classifier.fit(x_train, y_train, epochs=10, batch_size=64, verbose=1)









Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Epoch 1/10
50000/50000 [==============================] - 12s 247us/step - loss: 1.3298 - acc: 0.5430
Epoch 2/10
50000/50000 [==============================] - 11s 219us/step - loss: 0.8531 - acc: 0.6993
Epoch 3/10
50000/50000 [==============================] - 11s 219us/step - loss: 0.6305 - acc: 0.7796
Epoch 4/10
50000/50000 [==============================] - 11s 219us/step - loss: 0.4415 - acc: 0.8455
Epoch 5/10
50000/50000 [==============================] - 11s 219us/step - loss: 0.2973 - acc: 0.8953
Epoch 6/10
50000/50000 [==============================] - 11s 220us/step - loss: 0.2069 - acc: 0.9282
Epoch 7/10
50000/50000 [==============================] - 11s 220us/step - loss: 0.1676 - acc: 0.9412
Epoch 8/10
50000/50000 [==============================] - 11s 222us/step - loss: 0.1331 - acc: 0.9545
Epoch 9/10
50000/50000 [==============================] - 11s 222us/step - loss: 0

## 預測新圖片，輸入影像前處理要與訓練時相同
#### ((X-mean)/(std+1e-7) ):這裡的mean跟std是訓練集的
## 維度如下方示範

In [6]:
input_example=(np.zeros(shape=(1,32,32,3))-mean_train)/(std_train+1e-7) 
classifier.predict(input_example)

array([[5.8147951e-04, 2.4273200e-05, 1.7952069e-03, 6.8145263e-01,
        3.1115708e-01, 3.6730943e-04, 3.1538969e-03, 1.3552507e-04,
        1.2154283e-03, 1.1718275e-04]], dtype=float32)